# Technique - Binary Search

Binary search is a powerful array searching algorithm, but it's [notoriously tough to implement correctly](https://en.wikipedia.org/wiki/Binary_search_algorithm#Implementation_issues). I've tried a couple different approaches for this, but the best is just to memorize the one presented below (from [Wikipedia](https://en.wikipedia.org/wiki/Binary_search_algorithm#Procedure)). 

If `val` is absent, the exit condition is lo > hi, so we expect this to occur; by setting `lo`/`hi` to either `mid+1` or `mid-1` (rather than `mid` itself), we force `lo` and `hi` to intersect eventually. `val` can be absent in 3 ways:
- `val < arr[0]`
- `arr[-1] < val` 
- `arr[i] < val < arr[i+1]` for some `0 <= i < len(arr)-1` 

In [17]:
def binary_search(arr, val):
    lo, hi = 0, len(arr)-1

    while lo <= hi:
        mid = (hi+lo)//2
        if arr[mid] == val:
            return mid
        lo, hi = (mid+1, hi) if arr[mid] < val else (lo, mid-1)
    
    return -1 # returning lo gives correct insertion index

cases = [
    ([], 10, -1), # Empty arr
    ([10], 10, 0), # Singleton arr, val present
    ([0], 10, -1), # Singleton arr, val missing
    ([i for i in range(10)], 0, 0), # Even length array, value at lowest bound
    ([i for i in range(10)], 5, 5), # Even length array, value between bounds
    ([i for i in range(10)], 9, 9), # Even length array, value at highest bound    
    ([i for i in range(10)], 67, -1), # Even length array, value missing
    ([i for i in range(11)], 0, 0), # Odd length array, value at lowest bound
    ([i for i in range(11)], 6, 6), # Odd length array, value between bounds
    ([i for i in range(11)], 10, 10), # Odd length array, value at highest bound    
    ([i for i in range(11)], 67, -1), # Odd length array, value missing
]
for arr, val, expected in cases:
    actual = binary_search(arr, val)
    assert actual == expected, f"{arr},{val}: {expected} != {actual}"

## Example: [ Search a 2D Matrix II](https://leetcode.com/problems/search-a-2d-matrix-ii/)

We can leverage a binary search on each row for `O(m log(n))` performance. We can also prune out any rows or columns where `row[0] <= target <= row[-1]` does not hold, but since there are edge cases where we can prune only 1 or 2 out and we can't ensure we always prune out a certain factor of them, it's still `O(m log(n))`.

In [19]:
from typing import List

def binary_search(row, val):
    lo, hi = 0, len(row)-1
    while lo <= hi:
        mid = (hi+lo) // 2
        if row[mid] == val:
            return mid, True
        elif row[mid] < val:
            lo = mid + 1
        else:
            hi = mid - 1
    return lo, False

class Solution:
    def searchMatrix(self, matrix: List[List[int]], val: int) -> bool:
        # List construction here is O(n) and O(m), but we only do each
        # once regardless of input size. 
        lowest_row, found = binary_search([row[-1] for row in matrix], val)
        if found: return found
        highest_row, found = binary_search([row[0] for row in matrix], val)
        if found: return found
        
        # if lowest row == 0 and highest row == len(matrix)-1, this becomes
        # O(nlog(m)) for n rows and m columns. 
        for y in range(lowest_row, highest_row):
            _, found = binary_search(matrix[y], val)
            if found: return found

        return False

cases = [
    ([[1]], 1, True),
    ([[6]], 1, False),
    ([[1,4,7,11,15],[2,5,8,12,19],[3,6,9,16,22],[10,13,14,17,24],[18,21,23,26,30]],666, False), 
    ([[1,4,7,11,15],[2,5,8,12,19],[3,6,9,16,22],[10,13,14,17,24],[18,21,23,26,30]], 5, True),
]
s = Solution()
for matrix, val, expected in cases:
    actual = s.searchMatrix(matrix, val)
    assert actual == expected, f"{matrix,val}: {expected} != {actual}"

NameError: name 'List' is not defined

## Example: [Search in Rotated Sorted Array](https://leetcode.com/problems/search-in-rotated-sorted-array/)

## Example: [Kth missing positive number](https://leetcode.com/problems/kth-missing-positive-number/). 
There's a  O(n) solution where we enumerate every missing number until we find the k-th one. However, the array is strictly increasing, so `arr[i]-i-1` is the number of missing elements to the left of i, and we can use this to find the `k`-th missing:
```
     i =  [  0 1 2     3         4]
    arr = [  2,3,4,    7,       11]
missing = [1       5,6,  8,9,10,

i arr[i] (arr[i]-i-1) = left missing   
0   2      2-0-1 = 1
1   3      3-1-1 = 1
2   4      4-2-1 = 1
3   7      7-3-1 = 3
4   11    11-4-1 = 6

k   i     arr[i] - (left missing - k) - 1 = kth missing
5   4       11   - (   6   -       5) - 1 =   9     
1   0        2   - (   1   -       1) - 1 =   2


Note that because left missing = arr[i]-i-1:
kth missing = arr[i] - (left missing - k) - 1
            = arr[i] - (arr[i] - i - 1 - k) - 1 
            = arr[i] - arr[i] + i + 1 + k - 1
            = i + k
```
So with this, we can use a binary search to find the minimum `i` s.t. `k <= arr[i]-i-1`, from which we can then calculate the `k`-th missing. Note that the problem allows for `k` to fall outside of the array (`k=250` is valid for the above input), but we can handle this with a check before the binary search.

In [21]:
class Solution:
    def findKthPositive(self, arr, k):


sol = Solution()
arr = [2,3,4,7,11]
cases = [
    (arr, 7, 12),
    (arr, 6, 10),
    (arr, 5, 9),
    (arr, 4, 8),
    (arr, 3, 6),
    (arr, 2, 5),
    (arr, 1, 1),
]
for k, expected in cases:
    actual = sol.findKthPositive(arr, k)   
    assert actual == expected, f"{arr, k}: {expected} != {actual}"